# Deep-RL in gym using Keras

   
## Introduction and acknowledgments
This interactive notebook is an introduction on how to use Tensorflow (with the Keras API) to train a model under OpenAI's [gym](https://gym.openai.com/) scenarios. Below, I describe training and test loops that can be applied easily to any of the scenarios within gym. The pre-requisite knowledge is a basic understanding of coding and neural networks, as well as an understanding of Markoff decision processes, Q-learning, and reinforcement-learning (RL). The final algorithm is heavily based off of that described in [Tambet Matiisen's blog post](http://neuro.cs.ut.ee/demystifying-deep-reinforcement-learning/), which contains a pseudo-code outline of the algorithm as well as an excellent review of all pre-requisite knowledge. The batching and memory abstraction that I use is from the work of [Jannes Klaas](https://github.com/JannesKlaas/sometimes_deep_sometimes_learning/blob/master/reinforcement.ipynb), with slight modifications for compatibility.







## Dependencies
- Python 3.6 or higher
- NumPy
- Keras
- Tensorflow
- Gym

## Exploring gym
First, we import the libraries we will use throughout this notebook.

In [2]:
import gym

# Modeling import/export
import os.path
import json
from keras.models import model_from_json
from keras.models import load_model

# Data preprocessing and modeling
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import sgd




/Users/WilliamWu/miniconda3/envs/gym/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


The example I use for this tutorial is [Cartpole](https://gym.openai.com/envs/CartPole-v1/), a simple game in which our objective is to balance a moving pole on a cart on a one-dimensional track for as long as possible. For the sake of simplicity, we define a "win" to be whether the cart can successfully balance the pole for the full 200 timesteps, with a "loss" otherwise. To start, let's try running a couple of games of Cartpole, letting the computer decide what to do randomly.

In [3]:
env = gym.make('CartPole-v0')

# Playing 5 games of Cartpole
for game in range(5):
    initial_state = env.reset()
    # Each game lasts 200 timesteps
    for t in range(200):
        env.render()
        # Advance the game by a timestep using a random action
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done or t == 199:
            # Print the game results, then start the next game
            if t == 199:
                print("Game {}: Win".format(game + 1))
            else:
                print("Game {}: Loss after {} timesteps".format(game + 1, t + 1))
            break
    
env.close()
    

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Game 1: Loss after 12 timesteps
Game 2: Loss after 14 timesteps
Game 3: Loss after 13 timesteps
Game 4: Loss after 18 timesteps
Game 5: Loss after 15 timesteps


As we can see, each game is rather brief as the random strategy fares quite poorly. In order to learn more about the game so we can start building our model, we'll break down the loop step-by-step. 

In [4]:
env = gym.make('CartPole-v0')
initial_state = env.reset()
print(initial_state)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
[-0.02555043 -0.01205376  0.03941492 -0.0229813 ]


Here, we have created a new environment instance of Cartpole. As we can see, calling env.reset() returns an initial state represented as a vector of features. If we run the above codeblock multiple times, the output of the last line will be different every time, meaning that the initial state for Cartpole is random (although this is not necessarily the case for all of the scenarios within gym).

Let's try to advance the game by a timestep. First, we need to learn more about the action space of Cartpole.

In [5]:
# Finding the amount of different available actions
print("Size of the action space: {}".format(env.action_space.n))

Size of the action space: 2


In [6]:
# Sampling 5 random actions
for i in range(5):
    print("Action {}: {}".format(i + 1, env.action_space.sample()))

Action 1: 0
Action 2: 0
Action 3: 0
Action 4: 1
Action 5: 1


As we can see, the action space only consists of two possible actions, which are conveniently encoded as 0 and 1. Try going back to the previous loop in which we ran several games of Cartpole and changing the line `action = env.action_space.sample()`. Can you see which actions do the 0 and 1 encodings represent?

Now that we know a bit more about the action space of Cartpole, we can advance the game by calling env.step(action), which returns the following state represented as an array, the reward obtained by our action, a Boolean representing whether the game has reached a terminal state, and a dictionary containing metadata for the game, in that respective order. We ignore the dictionary returned within the array, as this information is primarily for debugging purposes and is not to be used for training. 

In [7]:
#Sampling a random action from that space
action = env.action_space.sample()
print(action)
observation, reward, done, info = env.step(action)
print(observation, reward, done)

0
[-0.02579151 -0.20771815  0.0389553   0.28187231] 1.0 False


Now that we know how to find an initial state, advance the state by a timestep, observe the resulting state and the acquired reward, and loop this process until we reach a terminal state, we have all of the information that we need to begin our process our implementing our deep Q-learning algorithm. The algorithm (as described within Matiisen's post) is shown below for your convenience.

```
do train-model:
    initialize replay memory D
    initialize action-value function Q with random weights
    observe initial state s
    repeat
        select an action a
            with probability ε select a random action
            otherwise select a = argmaxa’Q(s,a’)
        carry out action a
        observe reward r and new state s’
        store experience <s, a, r, s’> in replay memory D

        sample random transitions <ss, aa, rr, ss’> from replay memory D
        calculate target for each minibatch transition
            if ss’ is terminal state then tt = rr
            otherwise tt = rr + γmaxa’Q(ss’, aa’)
        train the Q network using (tt - Q(ss, aa))^2 as loss

        s = s'
    until terminated
```

## Implementing the algorithm
First, we need to define the replay memory. A sample implementation is provided below. Notice that `get_batch()` not only creates the input matrix at our desired batch size, but also creates the corresponding target vector as well. The values for the target vector are calculated as seen in the above pseudocode.

In [8]:
class Replay(object):
    """
    This code was taken, with very slight modification, from Jannes Klaas' deep RL example. I have
    preserved some of his annotations below, and added some of my own.
    link: https://github.com/JannesKlaas/sometimes_deep_sometimes_learning/blob/master/reinforcement.ipynb
    
    For reference, all starred (*) input arguments are hyperparameters to be adjusted.
    
    During gameplay all the experiences < s, a, r, s’ > are stored in a replay memory. 
    In training, batches of randomly drawn experiences are used to generate the input 
    and target for backpropagation.For this scenario, simple random sampling of experiences
    will suffice, but certain prioritized sampling strategies may prove useful elsewhere.
    """
    def __init__(self, env_dim, max_memory=100, discount=.9):
        """
        Inputs: 
        max_memory*: the maximum number of experiences we want to store
        discount*: the discount factor for future experience
        env_dim: the (fixed) length of the feature vector
        
        Output:
        A Replay object that stores experiences as a nested array. Each inner array contains the 
        < s, a, r, s’ > experiences as its first element, and a boolean value representing whether 
        the game has ended as its second element.
        
        [...
            [experience, game_over]
            [experience, game_over]
        ...]
        """
        
        self.max_memory = max_memory
        self.memory = list()
        self.discount = discount
        self.env_dim = env_dim

    def remember(self, states, game_over):
        """
        Saves a transition experience into memory
        
        Inputs:
        [s, a, r, s’]
        s: numpy array of features
        a: int encoding actions
        r: float representing reward
        s’: numpy array of features, same size as s
        
        game_over: boolean
        """
        
        #Save a state to memory
        self.memory.append([states, game_over])
        if len(self.memory) > self.max_memory:
            del self.memory[0]
            
    def get_batch(self, model, max_batch_size=10):
        """
        Sample from our memory and preprocess an input, target pair for training. 
        The Q-values for the target matrix are calculated according to the equation:
        Q(s, a) = r + gamma * max Q(s’,a’),
        where Q(s’,a’) is estimated using the model's weights upon the time of the method being called.
        
        Inputs:
        model: a Keras model that takes in a (m x self.env_dim) size matrix,
            where m is an undetermined row count, and outputs a (m x self.action_space.n) matrix
        max_batch_size*: maximum batch size
        
        Outputs:
        inputs: a numpy matrix of size (m x self.env_dim), which contains the feature vectors 
            for each state in the batch. m is either max_batch_size or the current length of 
            self.memory, whichever is smaller.
        targets: a numpy matrix of size (m x self.action_space.n), in which each row contains
            a vector of Q-Values for each action. The ith vector of Q-Values is to correspond to 
            the ith feature vector of the batch.
        
        
        """
        
        
        
        # Calculate the number of actions that can possibly be taken in the game
        # We found this value to be 2 earlier, but we can retrieve this value from our model once we build it
        num_actions = model.output_shape[-1]

        # If we have not yet saved enough states to memory, we must lower our batch size
        len_memory = len(self.memory)
        batch_size = min(len_memory, max_batch_size)
        
        # Initializing numpy matrices of proper sizes
        inputs = np.zeros((batch_size, self.env_dim))
        targets = np.zeros((batch_size, num_actions))
        
        # Random sampling of experiences from memory
        for i, idx in enumerate(np.random.randint(0, len_memory,
                                                  size=batch_size)):
            """
            Here we load one transition <s, a, r, s’> from memory
            state_t: initial state s
            action_t: action taken a
            reward_t: reward earned r
            state_tp1: the state that followed s’
            """
            state_t, action_t, reward_t, state_tp1 = self.memory[idx][0]
            
            # Checking whether the game had ended, as this affects our calculations of the Q-value
            game_over = self.memory[idx][1]
            # Copying the state feature vector over to input matrix at the appropriate row
            inputs[i:i+1] = state_t
            
            # First we fill the target values with the predictions of the model.
            # They will not be affected by training (since the training loss for them is 0)
            targets[i] = model.predict(state_t)[0]
            
            """
            If the game ended, the expected reward Q(s,a) should be the final reward r.
            Otherwise the target value is r + gamma * max Q(s’,a’)
            """
            #  Here Q_sa is max_a'Q(s', a')
            Q_sa = np.max(model.predict(state_tp1)[0])
            
            # if the game ended, the reward is the final reward
            if game_over:  # if game_over is True
                targets[i, action_t] = reward_t
            else:
                # r + gamma * max Q(s’,a’)
                targets[i, action_t] = reward_t + self.discount * Q_sa
        return inputs, targets
        

Next, we create our training loop. In deep RL, we train our model continually as we feed inputs and collect information from the game. Thus, we sample from our memory and run an iteration of backpropagation at each timestep. The resulting training algorithm looks very similar to the loop we used to run the game itself. As a result, we can actually render the game throughout the training process, which gives us the ability to visually monitor the training process and to adjust hyperparameters appropriately. 

In [9]:
def train(env, replayMemory, model, epochs, epsilon = .9, batch_size = 10,
          set_size = 10, timesteps = 200, output = 1, render = 0):
    """
    Run the game and train the model under the specified hyperparameters. During each frame, 
    we take an action according to the epsilon-greedy strategy. Rather than render the game
    at every step, we can choose to render the game periodically to improve the runtime of 
    the algorithm. We deem the collection of games that occur in between renders a "set", 
    and the size of a set can be adjusted as an argument of this function. 
    By default, the function will output the progress of the training after each set,
    through the average training loss per frame and the win count over the last set of games
    (which may not be indicative of the model's overall performance due to our 
    epsilon-greedy policy). 
    
    Inputs:
    env: a gym environment used and rendered for training
    replayMemory: an initialized Memory object for storing experiences and generating 
        training batches
    model: a Keras model with valid input and output sizes according to env
    epochs*: amount of games to play in the training cycle
    epsilon*: probability of exploration (vs. exploitation)
    batch_size*: size of batches used for training
    timesteps: length of each game
    set_size: amount of epochs in between rendered games
    output: enable/disable print output
    render: enable/disable rendering
    
    """
    
    
    # history arrays, useful for generating time series data
    set_hist = []
    win_hist = []
    
    # counters for tracking progress
    win_cnt = 0
    loss_over_set = 0.
    frames_over_set = 0
    
    # epochs := amount of games to run
    for i_episode in range(epochs):
        
        observation = env.reset()
        loss = 0.
        for t in range(timesteps):
            
            if i_episode % set_size == set_size - 1 and render == 1:
                env.render()
                
            # create a numpy array of the feature vector of the current state
            state_0 = np.array(observation, ndmin=2)
            
            # epsilon-greedy policy
            if np.random.rand() < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(model.predict(state_0)[0])
            
            # take a step and observe our reward and resulting state, while checking whether the game ended
            observation, reward, done, info = env.step(action)
            
            # create a numpy array of the feature vector of the resulting state
            state_1 = np.array(observation, ndmin=2)
            
            # store our experience, < s, a, r, s’ >
            replayMemory.remember([state_0, action, reward, state_1], done)
            
            # sample from our memory and create input and target matrices for training
            inputs, targets = replayMemory.get_batch(model, max_batch_size=batch_size)
            
            # train our model and record data
            batch_loss = model.train_on_batch(inputs, targets)
            loss += batch_loss
            
            frames_over_set += 1
            loss_over_set += batch_loss
            
            # if the game has ended, record data before moving on to the next game
            if done or t + 1 == timesteps:
                # note that a "win" here means reaching the last timestep
                if t + 1 == timesteps:
                    win_cnt += 1
                    win_hist.append(1)
                else:
                    win_hist.append(0)
                
                break
                
        # after each set, report progress and reset the counters
        if i_episode % set_size == set_size - 1:
            if output > 0:
                print("Episode finished after {} timesteps".format(t+1))
                print("Epoch {:03d}/{:03d} | Average Loss per frame over set: {:.4f} | Win count over past set: {}"
                      .format(i_episode + 1, epochs, loss_over_set / frames_over_set, win_cnt))
            
            win_cnt = 0
            loss_over_set = 0.
            frames_over_set = 0
            
            set_hist.append(win_hist)
            win_hist = []
                
            
        
    env.reset()
    env.close()

## Training
With our training function fully set up, we can begin to define hyperparameters and build our model. For this task, 3 simple dense layers will suffice.

In [41]:
"""
Hyperparameters
"""
epsilon = .10  # probability of choosing a random action instead of using the model to decide
max_memory = 200 # max number of experiences to be stored at once
hidden_size = 100 # size of the hidden layers within the network
batch_size = 20 # amount of experiences to sample into each batch for training
discount = .95 # value of future reward vs. current reward
learning_rate = .005 # the multiplicative rate at which the weights of the model are shifted
timesteps = 200 # length of each game (for Cartpole, ideally set this to between 100-200)
epochs = 300 # (Amount of games played)
set_size = 10 # rate at which games are rendered and progress is reported

In [42]:
env = gym.make('CartPole-v0')
feature_dims = len(env.reset())
num_actions = env.action_space.n
memory = Replay(max_memory = max_memory, discount = discount, env_dim = feature_dims)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [45]:
model = Sequential()
model.add(Dense(hidden_size, input_shape=(feature_dims,), activation='relu'))
model.add(Dense(hidden_size, activation='relu'))
model.add(Dense(num_actions))
model.compile(sgd(lr=learning_rate), "mse")
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 100)               500       
_________________________________________________________________
dense_20 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_21 (Dense)             (None, 2)                 202       
Total params: 10,802
Trainable params: 10,802
Non-trainable params: 0
_________________________________________________________________


With everything all set, we can finally train our model. With around 300 epochs, it takes around 30 minutes to fully train the model with a CPU.

In [46]:
train(env = env, replayMemory = memory, model = model, epochs = epochs,
      epsilon = epsilon, batch_size = batch_size, set_size = set_size, 
      timesteps = timesteps, output = 1, render = 0)

Episode finished after 9 timesteps
Epoch 010/300 | Average Loss per frame over set: 0.7186 | Win count over past set: 0
Episode finished after 11 timesteps
Epoch 020/300 | Average Loss per frame over set: 2.3550 | Win count over past set: 0
Episode finished after 8 timesteps
Epoch 030/300 | Average Loss per frame over set: 2.2142 | Win count over past set: 0
Episode finished after 9 timesteps
Epoch 040/300 | Average Loss per frame over set: 0.7991 | Win count over past set: 0
Episode finished after 10 timesteps
Epoch 050/300 | Average Loss per frame over set: 0.8686 | Win count over past set: 0
Episode finished after 10 timesteps
Epoch 060/300 | Average Loss per frame over set: 0.7768 | Win count over past set: 0
Episode finished after 14 timesteps
Epoch 070/300 | Average Loss per frame over set: 1.0704 | Win count over past set: 0
Episode finished after 15 timesteps
Epoch 080/300 | Average Loss per frame over set: 1.3015 | Win count over past set: 0
Episode finished after 12 timesteps

## Testing

Next, we define a testing function to test the performance of our model. The body of the function looks much like the loop where we ran Cartpole earlier, expect we use our model to decide our actions rather than `env.action_space.sample()`.

In [16]:
def test(model, games = 100, timesteps = 200, output = 1, render = 0):
    """
    Test our model and output results.
    
    Inputs:
    model: model to be tested
    games: amount of games to play
    timesteps: length of each game
    output: enable/disable print output
    render: enable/disable rendering 
    """
    
    Win_count = 0
    
    env = gym.make('CartPole-v0')
    for game in range(games):
        observation = env.reset()
        total_reward = 0
        for t in range(timesteps):
            if render == 1:
                env.render()
            state = np.array(observation, ndmin=2)
            action = np.argmax(model.predict(state)[0])
            observation, reward, done, info = env.step(action)
            total_reward += reward
            if done or t >= timesteps - 1:
                if t >= timesteps - 1:
                    Win_count += 1
                break
    if output == 1:
        print("Test results: {}/{} games won.".format(Win_count, games))
    env.close()
    

In [50]:
test(model, games = 10, render = 1, output= 1, timesteps = timesteps)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Test results: 10/10 games won.


With some tweaks to the hyperparameters, you should be able to reliably train a model that wins nearly every game. Typically, with a lower epsilon, it will take more epochs to train the model fully, but the success of the model will become more reliable. Because of the way that the training loop is structured, it is actually possible to further train the model after the initial epochs. If you find the model failing to win the game, try training the model under 50-100 additional games and rerunning the testing function.

## Saving the model
Once you've trained a successful model, Keras will allow you to easily save your model (both its structure and weights) with h5py. 

In [51]:
# Saving the model to your current working directory
pwd = os.path.abspath(".")
filename = "model"
model.save(os.path.join(pwd, filename + ".h5"))

## Final thoughts and appendix

Congratulations on building your first model for Cartpole! The code presented in this tutorial should be versatile enough to translate into a vast majority of the gym scenarios. When applying this design to other scenarios, modifications must be made to the win-tracking section of the training and test functions (and perhaps moving to a reward-based rather than win-based analysis of performance). For Cartpole, it isn't very difficult to train a model that can win 100% of the time, but, as you can see from the training output, the training isn't always reliable. Can you come up with model designs (or hyperparameters) that are able to train a winning model more reliably? Below are some questions for thought.

* Notice that when we build our deep RL model, we didn't have to actually find out what the values of the feature vector actually mean in terms of the game's state. Read the design of Cartpole [here](https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py). What parts of the game are described by the feature vector? Recall that our original goal was to design a model that could consistently balance the pole for 200 timesteps. Knowing that, what variables can we add to our feature vector that could improve our ability to model the scenario?

* You might have noticed that, in Cartpole, the reward assigned is exactly 1.0 in every timestep, as long as the pole doesn't tip over. Thus, what goal is our model (and learning algorithm) attempting to achieve? What changes can we make to our reward assignment to better achieve our actual goal? Will those changes in reward assignment require additional information from the feature vector?



### Further work
You can check out some extensions I made for this project in [appendix.py](./appendix.py)